In [10]:
# Remplace toutes les occurrences de ".." par "."
with open('Conso.csv', 'r', encoding='utf-8') as f:
    content = f.read()

content = content.replace('..', '.')

with open('Conso.csv', 'w', encoding='utf-8') as f:
    f.write(content)


In [ ]:
#Format heure local 

import csv
import pandas as pd
from datetime import timedelta

input_file = 'Conso.csv'
backup_file = 'Conso_backup.csv'
hours_to_add = 2

# 1) Detecter le séparateur
with open(input_file, 'r', encoding='utf-8', errors='replace') as f:
    sample = f.read(4096)
sniffer = csv.Sniffer()
try:
    dialect = sniffer.sniff(sample)
    delim = dialect.delimiter
except Exception:
    # fallback si sniff échoue
    delim = ';'

print(f"Détecté séparateur : {repr(delim)}")

# 2) Lire en texte brut pour préserver toutes les valeurs
df = pd.read_csv(input_file, sep=delim, dtype=str, keep_default_na=False, na_values=[], engine='python', encoding='utf-8')

# 3) Fonction qui décale date+time et remet en deux colonnes
def shift_date_time(df, col_date, col_time, hours=2):
    if col_date not in df.columns or col_time not in df.columns:
        print(f"Colonnes non trouvées : {col_date}, {col_time} — saut.")
        return 0

    # Concaténation (trim)
    combined = df[col_date].astype(str).str.strip() + ' ' + df[col_time].astype(str).str.strip()

    # Parser en datetime (dayfirst True car format DD/MM/YY), tolérant erreurs
    dt = pd.to_datetime(combined, dayfirst=True, format='%d/%m/%y %H:%M:%S', errors='coerce')

    # Compter les lignes valides
    valid_before = dt.notna().sum()
    if valid_before == 0:
        # essayer un parse plus permissif (sans format strict)
        dt = pd.to_datetime(combined, dayfirst=True, errors='coerce')

    valid_after = dt.notna().sum()
    if valid_after == 0:
        print(f"Aucune date/heure valide trouvée dans {col_date}/{col_time}.")
        return 0

    # Ajouter heures
    dt = dt + timedelta(hours=hours)

    # Remettre au format souhaité
    df.loc[dt.notna(), col_date] = dt[dt.notna()].dt.strftime('%d/%m/%y')
    df.loc[dt.notna(), col_time] = dt[dt.notna()].dt.strftime('%H:%M:%S')

    print(f"Décalage appliqué sur {col_date}/{col_time} : {valid_after} lignes modifiées.")
    return valid_after

# 4) Faire une sauvegarde
df.to_csv(backup_file, sep=delim, index=False, encoding='utf-8')
print(f"Sauvegarde créée : {backup_file}")

# 5) Appliquer le décalage sur RTC et GPS si présents
shift_date_time(df, 'RTC date', 'RTC time', hours=hours_to_add)
shift_date_time(df, 'GPS date', 'GPS time', hours=hours_to_add)

# 6) Écrire le fichier final (écrase Conso.csv)
df.to_csv(input_file, sep=delim, index=False, encoding='utf-8')
print(f"Fichier mis à jour : {input_file}")


Détecté séparateur : ';'
Sauvegarde créée : Conso_backup.csv
Décalage appliqué sur RTC date/RTC time : 164397 lignes modifiées.
Décalage appliqué sur GPS date/GPS time : 140676 lignes modifiées.
Fichier mis à jour : Conso.csv
